# 04-04 Async/Await

> 最优雅的异步写法：用同步的方式写异步代码

---

## 1. 基本语法

### async 函数

```javascript
// async 函数总是返回 Promise
async function fetchData() {
    return "数据";  // 相当于 return Promise.resolve("数据")
}

// 等价写法
function fetchData() {
    return Promise.resolve("数据");
}
```

### await 关键字

```javascript
// await 等待 Promise 完成，返回其结果
async function getUser() {
    const response = await fetch('/api/user');  // 等待请求完成
    const user = await response.json();          // 等待解析完成
    return user;
}

// 错误处理
async function getUserSafe() {
    try {
        const user = await fetchUser();
        return user;
    } catch (error) {
        console.error('获取失败:', error);
        return null;
    }
}
```

## 2. 对比三种写法

### 场景：获取用户信息，然后获取订单

In [ ]:
// ❌ 回调地狱
function getUserOrdersCallback(userId, callback) {
    db.getUser(userId, (err, user) => {
        if (err) return callback(err);
        db.getOrders(user.id, (err, orders) => {
            if (err) return callback(err);
            callback(null, { user, orders });
        });
    });
}

// ⚠️ Promise 链式
function getUserOrdersPromise(userId) {
    return db.getUser(userId)
        .then(user => {
            return db.getOrders(user.id)
                .then(orders => ({ user, orders }));
        });
}

// ✅ Async/Await
async function getUserOrders(userId) {
    const user = await db.getUser(userId);
    const orders = await db.getOrders(user.id);
    return { user, orders };
}

## 3. 错误处理模式

### 模式1：try/catch

In [ ]:
async function loadData() {
    try {
        const user = await fetchUser();
        const orders = await fetchOrders(user.id);
        return { user, orders };
    } catch (error) {
        console.error('加载失败:', error);
        throw error;  // 继续抛出或返回默认值
    }
}

### 模式2：逐个处理

In [ ]:
async function loadDataSafe() {
    let user = null;
    let orders = [];
    
    try {
        user = await fetchUser();
    } catch (e) {
        console.error('用户加载失败');
    }
    
    if (user) {
        try {
            orders = await fetchOrders(user.id);
        } catch (e) {
            console.error('订单加载失败');
        }
    }
    
    return { user, orders };
}

### 模式3：不中断的批量处理

In [ ]:
async function loadMultipleUsers(userIds) {
    const results = await Promise.allSettled(
        userIds.map(id => fetchUser(id))
    );
    
    const users = [];
    const errors = [];
    
    results.forEach((result, index) => {
        if (result.status === 'fulfilled') {
            users.push(result.value);
        } else {
            errors.push({ id: userIds[index], error: result.reason });
        }
    });
    
    return { users, errors };
}

## 4. 并行 vs 串行

### 串行执行（慢）

In [ ]:
// ❌ 串行：总时间 = A + B + C
async function loadSequential() {
    const a = await fetchA();  // 1s
    const b = await fetchB();  // 1s
    const c = await fetchC();  // 1s
    // 总时间：3秒
    return { a, b, c };
}

### 并行执行（快）

In [ ]:
// ✅ 并行：总时间 = max(A, B, C)
async function loadParallel() {
    const [a, b, c] = await Promise.all([
        fetchA(),  // 同时开始
        fetchB(),
        fetchC()
    ]);
    // 总时间：1秒
    return { a, b, c };
}

// ✅ 混合模式：有依赖关系时
async function loadMixed() {
    // 先并行加载用户和配置
    const [user, config] = await Promise.all([
        fetchUser(),
        fetchConfig()
    ]);
    
    // 再串行加载订单（依赖用户）
    const orders = await fetchOrders(user.id);
    
    return { user, config, orders };
}

## 5. 常见陷阱

### 陷阱1：忘记 await

In [ ]:
// ❌ 错误：忘记 await，返回的是 Promise
async function getData() {
    const result = fetchData();  // 忘记 await！
    console.log(result);  // Promise { <pending> }
    return result;
}

// ✅ 正确
async function getData() {
    const result = await fetchData();
    console.log(result);  // 实际数据
    return result;
}

### 陷阱2：在普通函数中使用 await

In [ ]:
// ❌ 错误：普通函数不能直接用 await
function normalFunction() {
    const data = await fetchData();  // SyntaxError!
}

// ✅ 解决1：改为 async 函数
async function asyncFunction() {
    const data = await fetchData();
}

// ✅ 解决2：使用 .then()
function normalFunction() {
    fetchData().then(data => {
        // 处理数据
    });
}

### 陷阱3：在 forEach 中使用 async

In [ ]:
// ❌ 错误：forEach 不会等待异步完成
async function processItems(items) {
    items.forEach(async (item) => {
        await processItem(item);  // 不会等待！
    });
    console.log('完成');  // 实际还没完成
}

// ✅ 解决1：使用 for...of
async function processItems(items) {
    for (const item of items) {
        await processItem(item);  // 会等待
    }
    console.log('完成');
}

// ✅ 解决2：使用 Promise.all
async function processItems(items) {
    await Promise.all(items.map(item => processItem(item)));
    console.log('完成');
}

## 6. 与Java的对比

| 特性 | Java | Node.js (Async/Await) |
|------|------|----------------------|
| 异步语法 | CompletableFuture | async/await |
| 错误处理 | exceptionally() | try/catch |
| 组合操作 | thenCompose, thenCombine | await + Promise.all |
| 超时控制 | orTimeout() | Promise.race |

### Java 等价写法

```java
// Java CompletableFuture
CompletableFuture<User> userFuture = fetchUser();
CompletableFuture<List<Order>> ordersFuture = userFuture
    .thenCompose(user -> fetchOrders(user.getId()));

// Node.js async/await
const user = await fetchUser();
const orders = await fetchOrders(user.id);
```

## 7. 实践练习

### 练习1：重构为 async/await

```javascript
// 将下面的 Promise 链重构为 async/await
function loadUserData(userId) {
    return db.findUser(userId)
        .then(user => {
            if (!user) throw new Error('用户不存在');
            return db.findOrders(user.id);
        })
        .then(orders => {
            return { orders, count: orders.length };
        })
        .catch(err => {
            console.error(err);
            return { orders: [], count: 0 };
        });
}
```

### 练习2：实现重试逻辑

```javascript
// 实现一个带指数退避的重试函数
// 第1次失败后等 1s，第2次等 2s，第3次等 4s...

async function retryWithBackoff(fn, maxRetries = 3) {
    // 你的代码
}

// 使用
const result = await retryWithBackoff(
    () => fetch('https://api.example.com/data'),
    3
);
```

### 练习3：并发控制

```javascript
// 实现一个限制并发数的 map 函数
// 类似 Promise.all，但最多同时执行 N 个

async function mapWithConcurrency(items, mapper, concurrency = 3) {
    // 你的代码
}

// 使用：同时最多3个请求
const results = await mapWithConcurrency(
    urls,
    url => fetch(url),
    3
);
```

---

## 本单元总结

你已经学习了：
1. ✅ 事件循环 - Node.js的并发核心
2. ✅ 回调函数 - 原始异步模式
3. ✅ Promise - 链式异步处理
4. ✅ Async/Await - 最现代的异步写法

**下一步：** 继续学习 [05-web-development](../05-web-development/) 或其他模块